In [1]:
import requests
from bs4 import BeautifulSoup
import re
from pprint import pprint

In [2]:
result = requests.get('https://comic.naver.com/webtoon/weekday.nhn')

soup = BeautifulSoup(result.content, 'lxml')
soup

#느리니까 SELENIUM 말고 뷰숲으로도 해보자.

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>네이버 만화 &gt; 요일별  웹툰 &gt; 전체웹툰</title>
<meta content="네이버 웹툰" property="og:title"/>
<meta content="https://ssl.pstatic.net/static/comic/images/og_tag_v2.png" property="og:image"/>
<meta content="매일매일 새로운 재미, 네이버 웹툰." property="og:description"/>
<meta content="https://comic.naver.com/webtoon/weekday.nhn" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<link href="/css/comic/common_20190605124420.css" rel="stylesheet" style="text/css"/>
<link href="/css/comic/comic_20190

In [3]:
week_cols = soup.select('div.col div.col_inner')

webtoon_dict = {}
for col in week_cols:
    weekday = col.find('span')
#     print(weekday.text)
    webtoons = col.select('ul li a')

    for webtoon in webtoons:
        title = webtoon.text
        link = webtoon['href']
        titleId = re.findall('[0-9]+',link)
#         print(titleId)               findall의 반환값은 list
        if not '\n' in title:
            webtoon_dict[title]  = {
                'title': title,
                'link': link,
                'title_id': titleId[0]
            }
pprint(webtoon_dict)

{'12차원 소년들': {'link': '/webtoon/list.nhn?titleId=717059&weekday=wed',
              'title': '12차원 소년들',
              'title_id': '717059'},
 '1초': {'link': '/webtoon/list.nhn?titleId=725586&weekday=fri',
        'title': '1초',
        'title_id': '725586'},
 '203호 저승사자': {'link': '/webtoon/list.nhn?titleId=670140&weekday=wed',
               'title': '203호 저승사자',
               'title_id': '670140'},
 '27-10': {'link': '/webtoon/list.nhn?titleId=725217&weekday=sun',
           'title': '27-10',
           'title_id': '725217'},
 '2인용 인간': {'link': '/webtoon/list.nhn?titleId=710769&weekday=sat',
            'title': '2인용 인간',
            'title_id': '710769'},
 '5kg을 위하여': {'link': '/webtoon/list.nhn?titleId=710639&weekday=sat',
              'title': '5kg을 위하여',
              'title_id': '710639'},
 '8월의 눈보라': {'link': '/webtoon/list.nhn?titleId=703856&weekday=sun',
             'title': '8월의 눈보라',
             'title_id': '703856'},
 'Doll 체인지': {'link': '/webtoon/list.nhn?titleId=6

- href는 a태그의 하이퍼링크 url이 들어가 있다.
- ~~그런데 href값을 보니 /부터 시작한다=html 아니다)
- '/~~'=리소스 경로! (?~~=쿼리 스트링)

                                 
                                 **결론**
                                 
- '프로토콜://주소값/' 가져와서 이거 뒤에 href값(리소스 경로+쿼리스트링)붙이자

- 어떤 프로토콜//주소값을 긁어와야되는지는 어떻게 알지?  => href 클릭해보면 알아

In [16]:
url_webtoon = 'http://comic.naver.com'

for idx, toon in enumerate(webtoon_dict):
    result = requests.get(url_webtoon + webtoon_dict[toon]['link'])
    soup = BeautifulSoup(result.content)
    
    current_href = soup.select('table.viewList tr td.title a')[0]['href']
    current_no = re.findall('no=([0-9]+)',current_href) # ->리스트 쓰지 말고 정규분포도
#     print(current_no)
    #;신의 탑':{title:,'link','titled':}에다가 'current_no'도 추가
       
    webtoon_dict[toon]['current_no'] = current_no[0]
        
    #진행 상황 체크(출력)
    
    if (idx+1)%40 == 0:
        print('{} 완료.. {}/{}'.format(toon, idx+1, len(webtoon_dict)))

마음의소리 완료.. 40/239
아도나이 완료.. 80/239
가담항설 완료.. 120/239
하르모니아 완료.. 160/239
고인의 명복 완료.. 200/239


In [7]:
pprint(webtoon_dict)

{'12차원 소년들': {'current_no': '39',
              'link': '/webtoon/list.nhn?titleId=717059&weekday=wed',
              'title': '12차원 소년들',
              'title_id': '717059'},
 '1초': {'current_no': '13',
        'link': '/webtoon/list.nhn?titleId=725586&weekday=fri',
        'title': '1초',
        'title_id': '725586'},
 '203호 저승사자': {'current_no': '84',
               'link': '/webtoon/list.nhn?titleId=670140&weekday=wed',
               'title': '203호 저승사자',
               'title_id': '670140'},
 '27-10': {'current_no': '14',
           'link': '/webtoon/list.nhn?titleId=725217&weekday=sun',
           'title': '27-10',
           'title_id': '725217'},
 '2인용 인간': {'current_no': '56',
            'link': '/webtoon/list.nhn?titleId=710769&weekday=sat',
            'title': '2인용 인간',
            'title_id': '710769'},
 '5kg을 위하여': {'current_no': '118',
              'link': '/webtoon/list.nhn?titleId=710639&weekday=sat',
              'title': '5kg을 위하여',
              'title_id': '710

In [6]:
title=input('웹툰명:')
titleid=webtoon_dict[title]['title_id']
current_no=webtoon_dict[title]['current_no']
page_no=1

웹툰명:신의 탑


In [10]:
comment_url='https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json'
#해당 페이지가 안읽어질 때 네트워크 들어가서 REQUEST URL긁어오기

#파라미터로 바꾸기
params={
     'ticket':'comic',
     'templateId':'webtoon',
     'pool':'cbox3',
     '_callback':'jQuery112406364821859830594_1560319818482',
     'lang':'ko',
     'country':'KR',
     'objectId':'{}_{}'.format(titleid,current_no), #titleid_current_no가 들어가 있음.
     'pageSize':15,
     'indexSize':10,
     'listType':'OBJECT',
     'pageType':'default',
     'page':page_no,
     'refresh':'true',
     'sort':'new',
     '_':1560319818488
}

#웹툰 하나만 긁어올거면 frame에서만 확인하면 됨.
#iframe 안에 들어와서 네트워크 clear, 변경하고 긁어오기. request한 것에 titleid와 current_no가 들어가 있다.-referer에 필요해서 긁어온거 
#url이 어떻게 생겼는지 먼저 규칙 확인, titleid와 current_no가 같음을 확인. 이정도는 해야 대량 정보를 긁어 올 수 있다.
    
                #FAIL이 뜨면? REQUEST HEADER 보자
        
        
headers={
    'Referer': 'https://comic.naver.com/comment/comment.nhn?titleId={}&no={}'.format(titleid, current_no)
}

result = requests.get(comment_url,headers=headers, params=params)
result.content #보통 <-얘는 HTML(계층적 태그 구조-a태그 지칭하면 정보얻음//단점:보기힘듦, 복잡 태그로 묶다 보니까 양이 크고 속도가 길다.) 
                #근데 아래 얘는 JSON  정보 구조화는 맞으나, tag대신 key를 지칭하면 정보를 얻을 수 있다.(딕셔너리와 유사)


b'jQuery112406364821859830594_1560319818482({"success":true,"code":"1000","message":"\xec\x9a\x94\xec\xb2\xad\xec\x9d\x84 \xec\x84\xb1\xea\xb3\xb5\xec\xa0\x81\xec\x9c\xbc\xeb\xa1\x9c \xec\xb2\x98\xeb\xa6\xac\xed\x95\x98\xec\x98\x80\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"183559_433","categoryId":"*","templateId":"default","commentNo":372022319,"parentCommentNo":372022319,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1560323599285,"contents":"\xec\x9a\xb0\xeb\xa6\xac\xec\xa7\x91\xea\xb0\x9c\xeb\x91\x90\xeb\xa7\x88\xeb\xa6\xac\xeb\x8f\x84 \xec\x8b\xa0\xec\x88\x98\xea\xb0\x80\xeb\xa5\xb4\xec\xb3\x90\xec\x95\xbc\xea\xb2\xa0\xeb\x8b\xa4.","userIdNo":"1ETL2","exposedUserIp":null,"lang":"ko","country":"KR","idType"

- requests.get( url, params )를 했을 때 result.content에 '실패'라고 뜬다면

1. network에 들어가서 요청 헤더(requests header)를 확인하자.  
2. 명시되어 있는 애들을 복사해서 headers에 dictionary형태로 넣는다.  
3. requests.get(url, headers)추가해서 요청하자.

In [11]:
result_update=re.findall('jQuery[0-9_(]+(.+?)\);',result.text)[0]
result_update

'{"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"183559_433","categoryId":"*","templateId":"default","commentNo":372022319,"parentCommentNo":372022319,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1560323599285,"contents":"우리집개두마리도 신수가르쳐야겠다.","userIdNo":"1ETL2","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"1등o신랑감S2","userProfileImage":"https://profile-phinf.pstatic.net/07467/24c982168f9450ff2c9d2e0278d895088e182a36eea95eecbf77580797f5cf9a.jpg","profileType":"naver","modTime":"2019-06-12T16:13:19+0900","modTimeGmt":"2019-06-12T07:13:19+0000","regTime":"2019-06-12T16:13:19+0900","regTimeGmt":"2019-06-12T07:13:19+0000","sympathyCount":0,"antipat

In [12]:
#result.content(json)->dictionary로 변환.
#JSON 안에서도 특정 댓글이 배당받아져있을거임
import json

result_json=json.loads(result_update)
result_json #'RESULT'=>'COMMENTLIST ->['CONTENTS']'

{'success': True,
 'code': '1000',
 'message': '요청을 성공적으로 처리하였습니다.',
 'lang': 'ko',
 'country': 'KR',
 'result': {'commentList': [{'ticket': 'comic',
    'objectId': '183559_433',
    'categoryId': '*',
    'templateId': 'default',
    'commentNo': 372022319,
    'parentCommentNo': 372022319,
    'replyLevel': 1,
    'replyCount': 0,
    'replyAllCount': 0,
    'replyPreviewNo': None,
    'replyList': None,
    'imageCount': 0,
    'imageList': None,
    'imagePathList': None,
    'imageWidthList': None,
    'imageHeightList': None,
    'commentType': 'txt',
    'stickerId': None,
    'sticker': None,
    'sortValue': 1560323599285,
    'contents': '우리집개두마리도 신수가르쳐야겠다.',
    'userIdNo': '1ETL2',
    'exposedUserIp': None,
    'lang': 'ko',
    'country': 'KR',
    'idType': 'naver',
    'idProvider': 'naver',
    'userName': '1등o신랑감S2',
    'userProfileImage': 'https://profile-phinf.pstatic.net/07467/24c982168f9450ff2c9d2e0278d895088e182a36eea95eecbf77580797f5cf9a.jpg',
    'profileType

In [13]:
result_json['result']


{'commentList': [{'ticket': 'comic',
   'objectId': '183559_433',
   'categoryId': '*',
   'templateId': 'default',
   'commentNo': 372022319,
   'parentCommentNo': 372022319,
   'replyLevel': 1,
   'replyCount': 0,
   'replyAllCount': 0,
   'replyPreviewNo': None,
   'replyList': None,
   'imageCount': 0,
   'imageList': None,
   'imagePathList': None,
   'imageWidthList': None,
   'imageHeightList': None,
   'commentType': 'txt',
   'stickerId': None,
   'sticker': None,
   'sortValue': 1560323599285,
   'contents': '우리집개두마리도 신수가르쳐야겠다.',
   'userIdNo': '1ETL2',
   'exposedUserIp': None,
   'lang': 'ko',
   'country': 'KR',
   'idType': 'naver',
   'idProvider': 'naver',
   'userName': '1등o신랑감S2',
   'userProfileImage': 'https://profile-phinf.pstatic.net/07467/24c982168f9450ff2c9d2e0278d895088e182a36eea95eecbf77580797f5cf9a.jpg',
   'profileType': 'naver',
   'modTime': '2019-06-12T16:13:19+0900',
   'modTimeGmt': '2019-06-12T07:13:19+0000',
   'regTime': '2019-06-12T16:13:19+0900',
 

In [14]:
result_json['result']['commentList']
len(result_json['result']['commentList'])

15

In [15]:
commentList=result_json['result']['commentList']
for comment in commentList:
    print(comment['contents'])

우리집개두마리도 신수가르쳐야겠다.
신탑은 쿠키 안지르고는 못배겨요 진짜..
2900번째!!!!
존재뮤ㅠㅠㅠㅠㅠㅠㅠㅠㅠ
나왜 저 원로가 화련인거같지
나도 쿠키지름 ㅌㅋ
얼마나 쿠키를 질러대면 순위가 떨어지냐 ㅋㅋㅋ
역대급 몰카로 이미 삼아제가 시작된거일수도
자전거 체인 끊고온다
이 집 말장난이 맛집이네 ㅋㅋㅋㅋㅋ 하-이 코믹 클라스
다른웹툰들은 독자들이 순위 신경도 안쓰는데 신의탑은 2위되자마자 이렇게 화력지원되는거 보니까 역시 팬층이 두껍긴 하네요ㅋㅋ 지금도 인기있지만 2013 2014 이때당시엔 진짜 웹툰계의 신드롬이었는데
2위뭐냐 신의탑이 자전거 웹툰보다 낮은게 말이 되나
난 인간을 포기하겟다 죠죠!!!!
라크도 떡상하쟈!!
남북거북이아니냐?
